# 🧬 A ProtT5 Expedition with Dr. Aris

Welcome back, researcher.

In our earlier journeys, we:
- **Listened to ESM-2** (a protein-native transformer trained at scale),
- **Consulted ProtBERT** (a BERT-style reader of amino-acid “text”).

Today, Dr. Aris invites a new scholar into the lab:

✅ **ProtT5** — a T5-family protein language model, known for learning rich representations that often transfer well to downstream biology tasks.

Our goal remains the same:
**Enzyme vs Non-Enzyme classification** using **ProtT5 embeddings** + lightweight classifiers (LR / SVM / MLP).

### ⚙️ Environment Setup

Before we open ProtT5’s manuscripts, we prepare the lab:
- import libraries once
- set random seeds
- confirm whether we are running on GPU (`cuda`) or CPU

In [8]:
# ================================
# Global Imports (Single Location)
# ================================

import os
import re
import math
import numpy as np
import pandas as pd
import torch
import time

from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel, T5EncoderModel

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    f1_score,
    accuracy_score
)

# ----------------
# Environment setup
# ----------------
os.environ["TOKENIZERS_PARALLELISM"] = "false"

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on:", device)

Running on: cuda


## 📂 Returning to the Library of Proteins

Dr. Aris retrieves the same curated collection we used before:
- load UniProt CSV
- clean sequences
- (optional) balance to a fixed benchmark size
- stratified train/test split

This keeps the comparison fair across ESM-2, ProtBERT, and now ProtT5.

In [2]:
# ==========================================
# Load stored UniProt CSV -> clean -> balance -> split
# ==========================================

DATA_PATH = "uniprot_swissprot_enzyme_nonenzyme_FULL.csv"
df = pd.read_csv(DATA_PATH)
print("Loaded:", DATA_PATH, " shape:", df.shape)

# ---- Detect column names robustly ----
seq_col = "Sequence" if "Sequence" in df.columns else "sequence"
len_col = "Length"   if "Length"   in df.columns else "length"
acc_col = "Entry" if "Entry" in df.columns else ("Accession" if "Accession" in df.columns else "accession")

# --------------------------
# Cleaning
# --------------------------
df[seq_col] = df[seq_col].astype(str).str.strip()
df = df[df[seq_col].notna() & (df[seq_col].str.len() > 0)].copy()

df[len_col] = pd.to_numeric(df[len_col], errors="coerce")
df = df[df[len_col].between(50, 1024)].copy()

if acc_col in df.columns:
    before = len(df)
    df = df.drop_duplicates(subset=[acc_col]).copy()
    print(f"De-duplicated by {acc_col}: {before} -> {len(df)}")

# Keep only standard amino acids (+X allowed)
allowed = set("ACDEFGHIKLMNPQRSTVWYX")
df = df[df[seq_col].apply(lambda s: set(s).issubset(allowed))].copy()

df.reset_index(drop=True, inplace=True)

print("\nAfter cleaning:", df.shape)
print("Label counts:\n", df["label"].value_counts())

# --------------------------
# Optional fixed-size balanced benchmark
# --------------------------
TOTAL_N = 40000  # set to None to keep all data
if TOTAL_N is not None:
    per_class = TOTAL_N // 2

    n_pos = min(per_class, (df["label"] == 1).sum())
    n_neg = min(per_class, (df["label"] == 0).sum())

    df_pos = df[df["label"] == 1].sample(n=n_pos, random_state=SEED)
    df_neg = df[df["label"] == 0].sample(n=n_neg, random_state=SEED)

    df = pd.concat([df_pos, df_neg], ignore_index=True)\
           .sample(frac=1.0, random_state=SEED)\
           .reset_index(drop=True)

    print("\nAfter balancing:", df.shape)
    print("Label counts:\n", df["label"].value_counts())

# --------------------------
# Train/Test split
# --------------------------
train_df, test_df = train_test_split(
    df,
    test_size=0.20,
    random_state=SEED,
    stratify=df["label"]
)

print("\nTrain:", train_df.shape, "Test:", test_df.shape)

train_df.head()

/scratch/local/25423975/ipykernel_218052/2353240010.py:6: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_PATH)


Loaded: uniprot_swissprot_enzyme_nonenzyme_FULL.csv  shape: (537346, 9)
De-duplicated by Entry: 537346 -> 537346

After cleaning: (536963, 9)
Label counts:
 label
0    270121
1    266842
Name: count, dtype: int64

After balancing: (40000, 9)
Label counts:
 label
0    20000
1    20000
Name: count, dtype: int64

Train: (32000, 9) Test: (8000, 9)


,Entry,Entry Name,Protein names,Organism,Length,EC number,Sequence,Reviewed,label
17005,P74737,RECA_SYNY3,Protein RecA (Recombinase A),Synechocystis sp. (strain ATCC 27184 / PCC 680...,354,NaN,MASTNISDREKALNAALAQIERSFGKGAIMRLGDATQMRVETISTG...,reviewed,0
19930,A1WXM7,SAHH_HALHL,Adenosylhomocysteinase (EC 3.13.2.1) (S-adenos...,Halorhodospira halophila (strain DSM 244 / SL1...,430,3.13.2.1,MSNQDYKVADISLADWGRKEIKIAESEMPGLMETRREFAAQKPLKG...,reviewed,1
8641,B7ZR65,SOX9A_XENLA,Transcription factor Sox-9-A,Xenopus laevis (African clawed frog),477,NaN,MNLLDPFMKMTEEQDKCMSGAPSPTMSDDSAGSPCPSGSGSDTENT...,reviewed,0
35026,P07435,OBP_BOVIN,Odorant-binding protein (OBP) (Olfactory mucos...,Bos taurus (Bovine),159,NaN,AQEEEAEQNLSELSGPWRTVYIGSTNPEKIQENGPFRTYFRELVFD...,reviewed,0
19704,P53690,MMP14_MOUSE,Matrix metalloproteinase-14 (MMP-14) (EC 3.4.2...,Mus musculus (Mouse),582,3.4.24.80,MSPAPRPSRSLLLPLLTLGTALASLGWAQGSNFSPEAWLQQYGYLP...,reviewed,1


## 🤖 Inviting ProtT5 to the Lab

ProtT5 is larger and heavier than ProtBERT.

So Dr. Aris does two things:
1. loads it in **half precision** on GPU (if available),
2. keeps **batch size small** to avoid CUDA out-of-memory.

Then we prepare sequences in the format ProtT5 expects:
- uppercase
- replace rare amino acids `U, Z, O, B → X`
- add spaces between amino acids (token-like format)

In [9]:
# ==========================================
# Load ProtT5 model + tokenizer (FIX)
# ==========================================

MODEL_NAME = "Rostlab/prot_t5_xl_uniref50"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

dtype = torch.float16 if device.type == "cuda" else torch.float32

# Encoder-only model (no decoder required)
try:
    model = T5EncoderModel.from_pretrained(MODEL_NAME, dtype=dtype).to(device)
except TypeError:
    model = T5EncoderModel.from_pretrained(MODEL_NAME, torch_dtype=dtype).to(device)

model.eval()

print("Loaded encoder:", MODEL_NAME)
print("Tokenizer fast?", getattr(tokenizer, "is_fast", False))
print("Model running on:", device, "| dtype:", dtype)

# ==========================================
# ProtT5 sequence preparation
# ==========================================

RARE_AA = re.compile(r"[UZOB]")

def prott5_prepare_sequence(seq: str) -> str:
    """
    Convert sequence to ProtT5 format:
    - Uppercase
    - Replace rare AAs with X
    - Space-separated tokens
    """
    seq = str(seq).strip().upper()
    seq = RARE_AA.sub("X", seq)
    return " ".join(list(seq))

Loaded encoder: Rostlab/prot_t5_xl_uniref50
Tokenizer fast? False
Model running on: cuda | dtype: torch.float16


## 🧠 When ProtT5 Begins to Read

Now the model reads each sequence and produces a hidden representation per token.

To turn token-level outputs into a single vector per protein, we use:
✅ **mean pooling** over valid (non-padding) tokens.

This yields embeddings shaped like:
- Train: `(N_train, hidden_dim)`
- Test : `(N_test, hidden_dim)`

In [10]:
# ==========================================
# Mean pooling helper
# ==========================================

@torch.no_grad()
def mean_pool(last_hidden_state: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
    mask = attention_mask.unsqueeze(-1).type_as(last_hidden_state)
    summed = (last_hidden_state * mask).sum(dim=1)
    counts = mask.sum(dim=1).clamp(min=1e-9)
    return summed / counts


# ==========================================
# Embedding extraction (ProtT5)
# ==========================================

@torch.no_grad()
def build_prott5_embeddings(sequences, batch_size=2, max_length=512) -> np.ndarray:
    prepared = [prott5_prepare_sequence(s) for s in sequences]
    loader = DataLoader(prepared, batch_size=batch_size, shuffle=False)

    all_embs = []
    for batch in loader:
        toks = tokenizer(
            list(batch),
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_length
        ).to(device)

        out = model(**toks)
        pooled = mean_pool(out.last_hidden_state, toks["attention_mask"])

        all_embs.append(pooled.float().cpu().numpy())  # ensure fp32 on CPU side

    return np.vstack(all_embs)

### Generate ProtT5 Embeddings

Dr. Aris will now:
- embed train proteins
- embed test proteins
- collect labels

If you hit CUDA memory issues:
- reduce `batch_size=2 → 1`
- or reduce `max_length=512 → 384`

In [11]:
# Detect sequence column (same logic as before)
seq_col_train = "Sequence" if "Sequence" in train_df.columns else "sequence"

# Build embeddings
X_train = build_prott5_embeddings(train_df[seq_col_train].tolist(), batch_size=2, max_length=512)
X_test  = build_prott5_embeddings(test_df[seq_col_train].tolist(), batch_size=2, max_length=512)

y_train = train_df["label"].to_numpy().astype(int)
y_test  = test_df["label"].to_numpy().astype(int)

print("Train embeddings:", X_train.shape)
print("Test embeddings :", X_test.shape)
print("Labels shape     :", y_train.shape, y_test.shape)

Train embeddings: (32000, 1024)
Test embeddings : (8000, 1024)
Labels shape     : (32000,) (8000,)


## 🧪 Probing ProtT5’s Knowledge

We reuse the same evaluation logic as ProtBERT:
- ROC-AUC
- PR-AUC
- F1
- Accuracy

So performance differences reflect the **embedding quality**, not metric changes.

In [12]:
def eval_binary_classifier(name, clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)

    # Probability-like scores for ROC/PR metrics
    if hasattr(clf, "predict_proba"):
        probs = clf.predict_proba(X_test)[:, 1]
    else:
        scores = clf.decision_function(X_test)
        probs = (scores - scores.min()) / (scores.max() - scores.min() + 1e-9)

    preds = (probs >= 0.5).astype(int)

    roc = roc_auc_score(y_test, probs)
    pr  = average_precision_score(y_test, probs)
    f1  = f1_score(y_test, preds)
    acc = accuracy_score(y_test, preds)

    print(f"{name:<14} ROC-AUC: {roc:.4f}")
    print(f"{name:<14} PR-AUC : {pr:.4f}")
    print(f"{name:<14} F1     : {f1:.4f}")
    print(f"{name:<14} Acc    : {acc:.4f}")
    print("-" * 60)

    return {"model": name, "roc_auc": roc, "pr_auc": pr, "f1": f1, "acc": acc}

### Train & Compare LR / SVM / MLP (ProtT5 Embeddings)

Now we perform the same downstream testing we used for ESM-2 and ProtBERT:
- **Logistic Regression** (strong linear baseline)
- **Linear SVM** (margin-based baseline, calibrated for probabilities)
- **MLP** (a small nonlinear head)

This keeps the comparison clean across models.

In [14]:
results = []

# ProtT5 + Logistic Regression
clf_lr = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(max_iter=3000, n_jobs=-1))
])
results.append(eval_binary_classifier("ProtT5+LR", clf_lr, X_train, y_train, X_test, y_test))

# ProtT5 + Linear SVM (calibrated)
base_svm = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", LinearSVC(max_iter=5000))
])
clf_svm = CalibratedClassifierCV(base_svm, method="sigmoid", cv=3)
results.append(eval_binary_classifier("ProtT5+SVM", clf_svm, X_train, y_train, X_test, y_test))

# ProtT5 + MLP
clf_mlp = Pipeline([
    ("scaler", StandardScaler()),
    ("mlp", MLPClassifier(
        hidden_layer_sizes=(256, 128),
        activation="relu",
        alpha=1e-4,
        max_iter=200,
        early_stopping=True,
        n_iter_no_change=10,
        random_state=SEED
    ))
])
results.append(eval_binary_classifier("ProtT5+MLP", clf_mlp, X_train, y_train, X_test, y_test))

pd.DataFrame(results)

ProtT5+LR      ROC-AUC: 0.9827
ProtT5+LR      PR-AUC : 0.9788
ProtT5+LR      F1     : 0.9459
ProtT5+LR      Acc    : 0.9459
------------------------------------------------------------


/apps/jupyter/6.5.4/lib/python3.10/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/apps/jupyter/6.5.4/lib/python3.10/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/apps/jupyter/6.5.4/lib/python3.10/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/apps/jupyter/6.5.4/lib/python3.10/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/apps/jupyter/6.5.4/lib/python3.10/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5

ProtT5+SVM     ROC-AUC: 0.9831
ProtT5+SVM     PR-AUC : 0.9791
ProtT5+SVM     F1     : 0.9471
ProtT5+SVM     Acc    : 0.9470
------------------------------------------------------------
ProtT5+MLP     ROC-AUC: 0.9946
ProtT5+MLP     PR-AUC : 0.9933
ProtT5+MLP     F1     : 0.9739
ProtT5+MLP     Acc    : 0.9739
------------------------------------------------------------


,model,roc_auc,pr_auc,f1,acc
0,ProtT5+LR,0.982745,0.978827,0.945909,0.945875
1,ProtT5+SVM,0.983118,0.979120,0.947106,0.947000
2,ProtT5+MLP,0.994554,0.993313,0.973872,0.973875


# 🧭 Reflections from the Expedition

Dr. Aris closes the ProtT5 manuscript gently.

What we achieved:
- We generated **ProtT5 embeddings** for enzyme vs non-enzyme proteins.
- We evaluated multiple lightweight classifiers (LR / SVM / MLP) using the **same metrics and split** as earlier notebooks.
- This makes it easy to compare representation quality across:
  **ESM-2 vs ProtBERT vs ProtT5**.

